In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
def remove_folder_contents(folder):
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                remove_folder_contents(file_path)
                os.rmdir(file_path)
        except Exception as e:
            print(e)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("secret_tokens")

!git clone https://github.com/kz-song/MMSRec.git

Cloning into 'MMSRec'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 82 (delta 14), reused 79 (delta 14), pack-reused 0
Unpacking objects: 100% (82/82), 18.82 MiB | 10.64 MiB/s, done.


In [3]:
%cd '/kaggle/working/MMSRec'
!sh weights/clip/download.sh
!pip install wandb easydict pynvml torchaudio decord jsonlines transformers torchvision

/kaggle/working/MMSRec
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ykj3x0ig
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ykj3x0ig
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 809.2 kB/s eta 0:00:000:00:01
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=24218a15bc6edb5c4f7cd1e4bd0f22d1f0c8bbbe9acc5955ab90611fab784595
  Stored in directory: /tmp/pip-ephem-wheel-cache-wquiapm5/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
--2024-04-23 03:31:41--  https://openaipublic.azureedge.net/clip/models/40d365715913c9da98579312b702a82c18be219cc2a73407c4526f58eba950af/ViT-B-32.pt
Resolving openaipublic.azureedge.net (openaipublic.azureedge.net)... 13.107.246.38, 13.107.213.38, 2620:1ec:bdf::

In [4]:
%cd '/kaggle/working/MMSRec/dataset/amazon/raw'

# Amazon All Beauty
!mkdir Beauty
%cd ./Beauty
!wget --no-check-certificate http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Beauty.csv
!wget --no-check-certificate http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Beauty.json.gz
%cd '/kaggle/working/MMSRec/dataset/amazon/preprocess'

/kaggle/working/MMSRec/dataset/amazon/raw
/kaggle/working/MMSRec/dataset/amazon/raw/Beauty
--2024-04-23 03:31:59--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Beauty.csv
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82432164 (79M) [text/csv]
Saving to: 'ratings_Beauty.csv'

ratings_Beauty.csv  100%[===================>]  78.61M  32.0MB/s    in 2.5s    

2024-04-23 03:32:01 (32.0 MB/s) - 'ratings_Beauty.csv' saved [82432164/82432164]

--2024-04-23 03:32:02--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Beauty.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99156148 (95M) [application/x-gzip]
Saving to: 'meta_Beauty.json.g

In [5]:
!python process_item.py

2024-04-23 03:32:15.235906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 03:32:15.236030: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 03:32:15.360874: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

-----Processing data ../raw/Beauty
load inter file ../raw/Beauty/ratings_Beauty.csv: 100%|█| 2023070/2023070 [00:05
Total inters: 2023070
load meta file ../raw/Beauty/meta_Beauty.json.gz: 259204it [00:36, 7134.88it/s]
Filter K core: user 5, item 5
	Filter: 2023070 inters to 394908 inters
	Filter: 394908 inters to 285290 inters
	Filter: 285290 inters to 235877 in

In [6]:
!python extract_features.py

2024-04-23 03:33:59.904001: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-23 03:33:59.904052: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-23 03:33:59.905579: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Processing data ../processed/Beauty----------
write info: 100%|█████████████████████| 12101/12101 [00:00<00:00, 116152.76it/s]


In [11]:
%cd '/kaggle/working/MMSRec'
!torchrun --nnodes=1:5 --nproc_per_node=1 --max_restarts=0 --rdzv_id=1 --rdzv_backend=c10d --rdzv_endpoint="127.0.0.1:12309" finetune_amazon.py --config="/kaggle/input/fintune-beauty-config-yaml/finetune_amazon.yaml"

/kaggle/working/MMSRec
[2024-04-23 03:50:23,747] torch.distributed.run: [WARNING] master_addr is only used for static rdzv_backend and when rdzv_endpoint is not specified.
04/23/2024 03:50:58 - INFO - __main__ - {'config': '/kaggle/input/fintune-beauty-config-yaml/finetune_amazon.yaml', 'wandb_enable': False, 'project_name': '', 'display_name': '', 'num_worker': 8, 'train_item_file': './dataset/amazon/processed/Beauty/item_feature.jsonl', 'train_seq_file': './dataset/amazon/processed/Beauty/train_seq.jsonl', 'eval_item_file': './dataset/amazon/processed/Beauty/item_feature.jsonl', 'eval_seq_file': './dataset/amazon/processed/Beauty/eval_seq.jsonl', 'test_item_file': './dataset/amazon/processed/Beauty/item_feature.jsonl', 'test_seq_file': './dataset/amazon/processed/Beauty/test_seq.jsonl', 'max_seq_length': 20, 'train_vision_format': 'embed', 'eval_vision_format': 'embed', 'test_vision_format': 'embed', 'max_vision_frames': 10, 'train_text_format': 'embed', 'eval_text_format': 'embed', 